In [20]:
import pandas as pd

history = pd.read_csv('data/unique_video_browsing_history_full_tiktok_data.csv')
history.sort_values(by=['Date'], inplace=True)

# Put the hashtags in lower case from history
history['Hashtags'] = history['Hashtags'].str.lower()

# Create an dataframe with which row being a single hashtag and the count of the hashtag with the columns names: Hashtag and Count
hashtags = history['Hashtags'].str.split(',', expand=True).stack().value_counts().rename_axis('Hashtag').reset_index(name='Count')

hashtags.head(10)

Hashtag  Count
0           #fyp  21735
1        #foryou   9952
2         #viral   7706
3    #foryoupage   7611
4          #edit   7347
5          #fypシ   6398
6        #batman   5114
7   #taylorswift   5059
8      #dccomics   5011
9            #dc   4836

In [21]:
#remove # from hashtags
hashtags['Hashtag'] = hashtags['Hashtag'].str.replace('#', '')

In [22]:
# Remove fyp

pattern = r'(fy\w+|fory\w+|fy|xyzbca|viral|trend\w+)'

# Remove row with re patterns from hashtags['Hashtags']
hashtags = hashtags[~hashtags['Hashtag'].str.contains(pattern)]

hashtags.head(10)


C:\Users\Test\AppData\Local\Temp\ipykernel_20796\2792701034.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  hashtags = hashtags[~hashtags['Hashtag'].str.contains(pattern)]


Hashtag  Count
4           edit   7347
6         batman   5114
7    taylorswift   5059
8       dccomics   5011
9             dc   4836
11   taylorswift   4211
12        marvel   3959
13    brucewayne   3922
14      swifttok   3149
16      erastour   2777

In [23]:
pattern1 = r'(tiktok|tok|toker)$'
pattern2 = r'^(tiktok|tok|toker)'

hashtags['Hashtag'] = hashtags['Hashtag'].str.replace(pattern1, '')
hashtags['Hashtag'] = hashtags['Hashtag'].str.replace(pattern2, '')


C:\Users\Test\AppData\Local\Temp\ipykernel_20796\2782663476.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  hashtags['Hashtag'] = hashtags['Hashtag'].str.replace(pattern1, '')
C:\Users\Test\AppData\Local\Temp\ipykernel_20796\2782663476.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  hashtags['Hashtag'] = hashtags['Hashtag'].str.replace(pattern2, '')


In [24]:
hashtags['Hashtag'] = hashtags['Hashtag'].str.replace(' ', '')

In [25]:
hashtags.head(10)

Hashtag  Count
4          edit   7347
6        batman   5114
7   taylorswift   5059
8      dccomics   5011
9            dc   4836
11  taylorswift   4211
12       marvel   3959
13   brucewayne   3922
14        swift   3149
16     erastour   2777

In [26]:
import wordninja as wn

# Create a new column with the hashtags splitted by words
hashtags['Hashtags_split'] = hashtags['Hashtag'].apply(lambda x: wn.split(x))
hashtags.reset_index(drop=True, inplace=True)

hashtags.head(10)

Hashtag  Count   Hashtags_split
0         edit   7347           [edit]
1       batman   5114         [batman]
2  taylorswift   5059  [taylor, swift]
3     dccomics   5011     [dc, comics]
4           dc   4836             [dc]
5  taylorswift   4211  [taylor, swift]
6       marvel   3959         [marvel]
7   brucewayne   3922   [bruce, wayne]
8        swift   3149          [swift]
9     erastour   2777     [eras, tour]

In [27]:

# Remove stop words
from nltk.corpus import stopwords

stop = stopwords.words('english')
hashtags['Hashtags_split'] = hashtags['Hashtags_split'].apply(lambda x: [item for item in x if item not in stop])

hashtags.head(10)


Hashtag  Count   Hashtags_split
0         edit   7347           [edit]
1       batman   5114         [batman]
2  taylorswift   5059  [taylor, swift]
3     dccomics   5011     [dc, comics]
4           dc   4836             [dc]
5  taylorswift   4211  [taylor, swift]
6       marvel   3959         [marvel]
7   brucewayne   3922   [bruce, wayne]
8        swift   3149          [swift]
9     erastour   2777     [eras, tour]

In [28]:
from collections import defaultdict

# Create a dictionary to aggregate counts
hashtags_counts = defaultdict(int)
hashtags_list = hashtags["Hashtag"]

for i in range(len(hashtags)):
    key = tuple(hashtags['Hashtags_split'][i])

    # Check if the Hashtags_split lists are exactly the same
    if key in hashtags_counts:
        hashtags_counts[key] += hashtags['Count'][i]
        hashtags_list = hashtags_list.drop(i)
    else:
        hashtags_counts[key] = hashtags['Count'][i]


# Create a new DataFrame from the aggregated counts
aggregated_data = {
    'Hashtag': hashtags_list,
    'Hashtags_split': [list(key) for key in hashtags_counts.keys()],
    'Count': list(hashtags_counts.values())
}

aggregated_df = pd.DataFrame(aggregated_data)

# Display the aggregated DataFrame
print(aggregated_df)


                             Hashtag                      Hashtags_split  \
0                               edit                              [edit]   
1                             batman                            [batman]   
2                        taylorswift                     [taylor, swift]   
3                           dccomics                        [dc, comics]   
4                                 dc                                [dc]   
...                              ...                                 ...   
86904                smallartisthelp               [small, artist, help]   
86905                     artsupport                      [art, support]   
86906  anthonymackieandsebastianstan  [anthony, mackie, sebastian, stan]   
86908                   wyldeflowers                    [wylde, flowers]   
86909                      slashfans                       [slash, fans]   

       Count  
0       7762  
1       7533  
2       9715  
3       5422  
4       5809

In [33]:
#sort table by hashtag 
import re
aggregated_df = aggregated_df.sort_values(by=['Hashtag'])

#delete empty hashtags and hashtags not in the english abc but keep numbers
aggregated_df = aggregated_df[aggregated_df['Hashtag'] != '']
english_alphabet_pattern = re.compile(r'^[a-zA-Z0-9]+$')
aggregated_df = aggregated_df[aggregated_df['Hashtag'].str.match(english_alphabet_pattern)]

aggregated_df.to_csv('data/aggregated_hashtags.csv', index=False)

In [153]:
from sklearn.metrics import jaccard_score
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

# Calculate Jaccard similarity between hashtags
def jaccard_similarity(set1, set2):
    if len(set1 | set2) == 0:
        return 0
    return len(set1 & set2) / len(set1 | set2)

similarities = []
hashtags_list = aggregated_df['Hashtags_split'][70000:].tolist()
for i in range(len(hashtags_list)):
    row = []
    for j in range(len(hashtags_list)):
        similarity = jaccard_similarity(set(hashtags_list[i]), set(hashtags_list[j]))
        row.append(similarity)
    similarities.append(row)


similarity_matrix = pd.DataFrame(similarities, columns=aggregated_df['Hashtag'][70000:], index=aggregated_df['Hashtag'][70000:])

C:\Users\Test\AppData\Local\Temp\ipykernel_20796\621681451.py:11: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  hashtags_list = aggregated_df['Hashtags_split'][70000:].tolist()
C:\Users\Test\AppData\Local\Temp\ipykernel_20796\621681451.py:20: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  similarity_matrix = pd.DataFrame(similarities, columns=aggregated_df['Hashtag'][70000:], index=aggregated_df['Hashtag'][70000:])


In [154]:
similarity_matrix

Hashtag                winniethepoohfan  winniethepoohmpreg  \
Hashtag                                                       
winniethepoohfan                    1.0                 0.4   
winniethepoohmpreg                  0.4                 1.0   
winniethepoohtheme                  0.5                 0.4   
winning                             0.0                 0.0   
winningisnteverything               0.0                 0.0   
...                                 ...                 ...   
zyxbca                              0.0                 0.0   
zyxcba                              0.0                 0.0   
zzavid                              0.0                 0.0   
zzavidedit                          0.0                 0.0   
zzz                                 0.0                 0.0   

Hashtag                winniethepoohtheme   winning  winningisnteverything  \
Hashtag                                                                      
winniethepoohfan                      0.5  0.000000               0.000000   
winniethepoohmpreg                    0.4  0.000000               0.000000   
winniethepoohtheme                    1.0  0.000000               0.000000   
winning                               0.0  1.000000               0.333333   
winningisnteverything                 0.0  0.333333               1.000000   
...                                   ...       ...                    ...   
zyxbca                                0.0  0.000000               0.000000   
zyxcba                                0.0  0.000000               0.000000   
zzavid                                0.0  0.000000               0.000000   
zzavidedit                            0.0  0.000000               0.000000   
zzz                                   0.0  0.000000               0.000000   

Hashtag                winnipeg  winnythebish  winonameadows  winonaryder  \
Hashtag                                                                     
winniethepoohfan            0.0           0.0            0.0          0.0   
winniethepoohmpreg          0.0           0.0            0.0          0.0   
winniethepoohtheme          0.0           0.0            0.0          0.0   
winning                     0.0           0.0            0.0          0.0   
winningisnteverything       0.0           0.0            0.0          0.0   
...                         ...           ...            ...          ...   
zyxbca                      0.0           0.0            0.0          0.0   
zyxcba                      0.0           0.0            0.0          0.0   
zzavid                      0.0           0.0            0.0          0.0   
zzavidedit                  0.0           0.0            0.0          0.0   
zzz                         0.0           0.0            0.0          0.0   

Hashtag                winston  ...  zxycba      zyan  zycxba  zylerthecat  \
Hashtag                         ...                                          
winniethepoohfan           0.0  ...    0.00  0.000000     0.0          0.0   
winniethepoohmpreg         0.0  ...    0.00  0.000000     0.0          0.0   
winniethepoohtheme         0.0  ...    0.00  0.000000     0.0          0.0   
winning                    0.0  ...    0.00  0.000000     0.0          0.0   
winningisnteverything      0.0  ...    0.00  0.000000     0.0          0.0   
...                        ...  ...     ...       ...     ...          ...   
zyxbca                     0.0  ...    0.00  0.333333     0.2          0.2   
zyxcba                     0.0  ...    0.25  0.333333     0.2          0.2   
zzavid                     0.0  ...    0.00  0.000000     0.0          0.0   
zzavidedit                 0.0  ...    0.00  0.000000     0.0          0.0   
zzz                        0.0  ...    0.00  0.000000     0.0          0.0   

Hashtag                zyvbca  zyxbca  zyxcba    zzavid  zzavidedit  zzz  
Hashtag                                                                  

In [ ]:
# Failed attempt - not in use
from scipy.cluster.hierarchy import linkage, fcluster
import pandas as pd

# Assuming you already have the similarity_matrix DataFrame

# Convert the similarity matrix to a condensed distance matrix
distance_matrix = 1 - similarity_matrix

# Perform hierarchical clustering
linkage_matrix = linkage(distance_matrix, method='average')

# Set a threshold for forming clusters
threshold = 0.99 # Adjust as needed
clusters = fcluster(linkage_matrix, threshold, criterion='distance')

sample_fifty = aggregated_df[:1000]

# Add the cluster labels to the DataFrame
sample_fifty['Cluster'] = clusters

# Display the DataFrame with cluster labels
print(sample_fifty[['Hashtag', 'Cluster']])

In [155]:
import pandas as pd

# Assuming you already have the similarity_matrix DataFrame

# Set a similarity threshold
threshold = 0.35 # Adjust as needed
# 0.49 is good
# 0.1 is bad
# 0.5 is also bad
# 0.3 has cons and pros

# Initialize an empty list to store clusters
clusters = []

# Iterate through the similarity matrix and create clusters
for i in range(len(similarity_matrix)):
    cluster = [similarity_matrix.index[i]]  # Start a new cluster with the current hashtag
    for j in range(i + 1, len(similarity_matrix)):
        if similarity_matrix.iloc[i, j] > threshold:
            cluster.append(similarity_matrix.index[j])  # Add the hashtag to the cluster
    clusters.append(cluster)

sample_fifty = aggregated_df[70000:]

# Add the cluster labels to the DataFrame
sample_fifty['Cluster'] = clusters
# 
# Display the DataFrame with cluster labels
print(sample_fifty[['Hashtag', 'Cluster']])

# Create a DataFrame with cluster labels
# cluster_df = pd.DataFrame({'Hashtag': similarity_matrix.index, 'Cluster': clusters})

# Merge the cluster information with the sample_fifty DataFrame
# sample_fifty = pd.merge(sample_fifty, cluster_df, on='Hashtag', how='left')

# Display the DataFrame with cluster labels
# print(sample_fifty[['Hashtag', 'Cluster']])

                     Hashtag  \
13155       winniethepoohfan   
38247     winniethepoohmpreg   
54115     winniethepoohtheme   
24858                winning   
83394  winningisnteverything   
...                      ...   
38279                 zyxbca   
343                   zyxcba   
8622                  zzavid   
21916             zzavidedit   
71202                    zzz   

                                                 Cluster  
13155  [winniethepoohfan, winniethepoohmpreg, winniet...  
38247           [winniethepoohmpreg, winniethepoohtheme]  
54115                               [winniethepoohtheme]  
24858                                          [winning]  
83394                            [winningisnteverything]  
...                                                  ...  
38279                                   [zyxbca, zyxcba]  
343                                             [zyxcba]  
8622                                [zzavid, zzavidedit]  
21916                    

C:\Users\Test\AppData\Local\Temp\ipykernel_20796\1993009964.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_fifty['Cluster'] = clusters


In [156]:
# put the count of every row in sample_fifty to zero
sample_fifty['Count'] = 0

C:\Users\Test\AppData\Local\Temp\ipykernel_20796\3290729637.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_fifty['Count'] = 0


In [157]:
sample_fifty.head(10)

Hashtag               Hashtags_split  Count  \
13155       winniethepoohfan          [winnie, pooh, fan]      0   
38247     winniethepoohmpreg      [winnie, pooh, mp, reg]      0   
54115     winniethepoohtheme        [winnie, pooh, theme]      0   
24858                winning                    [winning]      0   
83394  winningisnteverything  [winning, isnt, everything]      0   
62281               winnipeg                   [winnipeg]      0   
26621           winnythebish            [win, ny, b, ish]      0   
21917          winonameadows            [winona, meadows]      0   
33351            winonaryder              [winona, ryder]      0   
11679                winston                    [winston]      0   

                                                 Cluster  
13155  [winniethepoohfan, winniethepoohmpreg, winniet...  
38247           [winniethepoohmpreg, winniethepoohtheme]  
54115                               [winniethepoohtheme]  
24858                                          [winning]  
83394                            [winningisnteverything]  
62281                                         [winnipeg]  
26621                                     [winnythebish]  
21917                                    [winonameadows]  
33351                                      [winonaryder]  
11679  [winston, winstonandnick, winstonbishop, winst...

In [158]:
for i in range(0,len(sample_fifty['Cluster'].tolist())):
    for j in sample_fifty['Cluster'].tolist()[i]:
        print(j)
        if j in aggregated_df['Hashtag'].tolist():
            count_to_add = aggregated_df[aggregated_df['Hashtag'] == j]['Count'].tolist()[0]
            print(count_to_add)
            # Update the count in sample_fifty DataFrame using iloc
            sample_fifty.iloc[i, sample_fifty.columns.get_loc('Count')] += count_to_add
            print(sample_fifty.iloc[i, sample_fifty.columns.get_loc('Count')])
        


winniethepoohfan
5
5
winniethepoohmpreg
2
7
winniethepoohtheme
1
8
winniethepoohmpreg
2
2
winniethepoohtheme
1
3
winniethepoohtheme
1
1
winning
2
2
winningisnteverything
1
1
winnipeg
1
1
winnythebish
2
2
winonameadows
3
3
winonaryder
2
2
winston
6
6
winstonandnick
1
7
winstonbishop
16
23
winstondog
2
25
winstonduke
2
27
winstonedit
1
28
winstongraham
1
29
winstonschmidt
3
32
winstonandcecemessaround
2
2
winstonandnick
1
1
winstonbishop
16
16
winstonbishopedits
1
17
winstonbishopedits
1
1
winstondog
2
2
winstonduke
2
2
winstonedit
1
1
winstonschmidtedit
1
2
winstongraham
1
1
winstonnbchannibal
1
1
winstonnewgirl
1
1
winstonschmidt
3
3
winstonschmidtedit
1
4
winstonschmidtedit
1
1
winter
9
9
winterbaron
5
14
winterbones
1
15
winterfalcon
13
28
winterfell
3
31
winterfield
1
32
winterfrost
5
37
wintergames
1
38
winterhawk
2
40
winteriron
3
43
winteriscoming
1
44
winterpanther
1
45
wintersoldier
128
173
wintersoldieredit
10
183
winterspider
1
184
wintervibes
1
185
winterwidow
4
189
winterwi

In [159]:
clusters = sample_fifty[['Hashtag', 'Cluster', 'Count']]
clusters.to_csv('data/hashtags_clusters.csv', index=False)

In [160]:
clusters = pd.read_csv('data/hashtags_clusters.csv')

In [161]:
# order the clusters dataframe by the number of hashtags in each cluster
clusters_asc = pd.read_csv('data/hashtags_clusters.csv')
clusters_desc = pd.read_csv('data/hashtags_clusters.csv')

clusters_asc['Length'] = clusters_asc['Cluster'].str.split(',').str.len()
clusters_desc['Length'] = clusters_desc['Cluster'].str.split(',').str.len()

clusters_asc.sort_values(by=['Length'], inplace=True, ascending=True)
clusters_desc.sort_values(by=['Length'], inplace=True, ascending=False)
clusters_asc.drop(columns=['Length'], inplace=True)
clusters_desc.drop(columns=['Length'], inplace=True)

# clusters.head(10)


In [162]:
# get the row outside of the list
clusters_desc['Cluster'] = clusters_desc['Cluster'].str.replace('[', '')
clusters_desc['Cluster'] = clusters_desc['Cluster'].str.replace(']', '')
clusters_desc['Cluster'] = clusters_desc['Cluster'].str.replace(" ", '')

clusters_asc['Cluster'] = clusters_asc['Cluster'].str.replace('[', '')
clusters_asc['Cluster'] = clusters_asc['Cluster'].str.replace(']', '')
clusters_asc['Cluster'] = clusters_asc['Cluster'].str.replace(" ", '')

C:\Users\Test\AppData\Local\Temp\ipykernel_20796\1569565536.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  clusters_desc['Cluster'] = clusters_desc['Cluster'].str.replace('[', '')
C:\Users\Test\AppData\Local\Temp\ipykernel_20796\1569565536.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  clusters_desc['Cluster'] = clusters_desc['Cluster'].str.replace(']', '')
C:\Users\Test\AppData\Local\Temp\ipykernel_20796\1569565536.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  clusters_asc['Cluster'] = clusters_asc['Clus

In [163]:
indexes_to_drop = []
sub_indexes = []

for index, row in clusters_asc.iterrows():
    mask = clusters_desc['Cluster'].apply(lambda x: set(row['Cluster'].split(',')).issubset(set(x.split(','))))
    mask[index] = False  # Exclude the same index
    matching_clusters = clusters_desc[mask]

    if not matching_clusters.empty:
        print(row['Cluster'])
        print(index)
        sub_indexes.append(matching_clusters['Cluster'].index.tolist()[0])
        indexes_to_drop.append(index)

'xiuxialing'
823
'xfactor'
813
'zachsangshow'
1466
'xforce'
815
'xman'
827
'zachbia'
1459
'xmen24hdrive'
836
'zachbryan'
1460
'zachkornfeld'
1463
'xmaspresents'
829
'xenkthepaladin'
807
'xmenaserieanimada'
843
'zachwellison'
1468
'xbox360'
781
'xavierstrilogy'
776
'xaviersschool'
775
'xboxbarbie'
782
'xavieredit'
772
'zacksnydersjusticeleague'
1477
'xanilla'
769
'xanderwildeedit'
768
'xavieredits'
773
'xboxcontroller'
785
'zackaddybones'
1470
'xboxshowcase'
795
'zackbiaoliviarodrigo'
1473
'xboxone'
792
'xboxmemes'
791
'xboxmeme'
790
'xboxgames'
787
'zackman2021'
1475
'zachandmaggie'
1455
'xodaylight'
910
'xochitlgomez'
909
'xmovie'
905
'xoffice'
911
'xmfc'
904
'zzavidedit'
1803
'yowzah'
1401
'xmenthelaststandedit'
898
'xmentas'
896
'xofswords'
913
'youtubersen'
1392
'xqcdrama'
928
'youtuberson'
1393
'xoxogossipgirl'
923
'youtubeshorts'
1394
'xotwod'
920
'xokittyseries'
918
'youtubeworkout'
1395
'yoyoktaylorswift'
1404
'yoyoyo'
1405
'yurispiderman'
1435
'yuriwatanabe'
1436
'xmendofpedit

In [164]:
for index in sub_indexes:
    clusters.iloc[index, clusters.columns.get_loc('Count')] += clusters.iloc[indexes_to_drop[0], clusters.columns.get_loc('Count')]

In [165]:
for index in indexes_to_drop:
    clusters.drop(index, inplace=True)

In [166]:
clusters.to_csv('data/clean_hashtags/hashtags_clusters_clean_70k_to_end_sort.csv', index=False)

In [170]:
#join all the clusters csv files
import glob

path = r'data/clean_hashtags_sort' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None)
    li.append(df)

hashtags_clusters = pd.concat(li, axis=0, ignore_index=True)

hashtags_clusters.to_csv('data/all_hashtags_clusters_sort.csv', index=False)



In [173]:
#remove the hashtags from aggregated_df that are not in hashtags_clusters
hashtags_clusters = pd.read_csv('data/all_hashtags_clusters_sort.csv')
aggregated_df_new = aggregated_df[aggregated_df['Hashtag'].isin(hashtags_clusters['Hashtag'].tolist())]


In [174]:
len(aggregated_df_new)

37765

In [175]:
from sklearn.metrics import jaccard_score
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

# Calculate Jaccard similarity between hashtags
def jaccard_similarity(set1, set2):
    if len(set1 | set2) == 0:
        return 0
    return len(set1 & set2) / len(set1 | set2)

similarities = []
hashtags_list = aggregated_df_new['Hashtags_split'].tolist()
for i in range(len(hashtags_list)):
    row = []
    for j in range(len(hashtags_list)):
        similarity = jaccard_similarity(set(hashtags_list[i]), set(hashtags_list[j]))
        row.append(similarity)
    similarities.append(row)


similarity_matrix = pd.DataFrame(similarities, columns=aggregated_df_new['Hashtag'], index=aggregated_df_new['Hashtag'])

In [176]:
import pandas as pd

# Assuming you already have the similarity_matrix DataFrame

# Set a similarity threshold
threshold = 0.35 # Adjust as needed
# 0.49 is good
# 0.1 is bad
# 0.5 is also bad
# 0.3 has cons and pros

# Initialize an empty list to store clusters
clusters = []

# Iterate through the similarity matrix and create clusters
for i in range(len(similarity_matrix)):
    cluster = [similarity_matrix.index[i]]  # Start a new cluster with the current hashtag
    for j in range(i + 1, len(similarity_matrix)):
        if similarity_matrix.iloc[i, j] > threshold:
            cluster.append(similarity_matrix.index[j])  # Add the hashtag to the cluster
    clusters.append(cluster)

sample_fifty = aggregated_df_new

# Add the cluster labels to the DataFrame
sample_fifty['Cluster'] = clusters
# 
# Display the DataFrame with cluster labels
print(sample_fifty[['Hashtag', 'Cluster']])

           Hashtag               Cluster
33891            0           [0, d0nald]
83767         0001                [0001]
29751        000u4  [000u4, 4u, edits4u]
73700          001                 [001]
34404          007                 [007]
...            ...                   ...
3106        zxycba              [zxycba]
3623        zycxba              [zycxba]
38309  zylerthecat         [zylerthecat]
8622        zzavid              [zzavid]
71202          zzz                 [zzz]

[37765 rows x 2 columns]


C:\Users\Test\AppData\Local\Temp\ipykernel_20796\2040321098.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_fifty['Cluster'] = clusters


In [177]:
sample_fifty.to_csv('sample_fifty.csv', index=False)

In [178]:
sample_fifty['Count'] = 0

C:\Users\Test\AppData\Local\Temp\ipykernel_20796\1442908076.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_fifty['Count'] = 0


In [179]:
for i in range(0,len(sample_fifty['Cluster'].tolist())):
    for j in sample_fifty['Cluster'].tolist()[i]:
        print(j)
        if j in aggregated_df['Hashtag'].tolist():
            count_to_add = aggregated_df[aggregated_df['Hashtag'] == j]['Count'].tolist()[0]
            print(count_to_add)
            # Update the count in sample_fifty DataFrame using iloc
            sample_fifty.iloc[i, sample_fifty.columns.get_loc('Count')] += count_to_add
            print(sample_fifty.iloc[i, sample_fifty.columns.get_loc('Count')])

0
2
2
d0nald
1
3
0001
1
1
000u4
3
3
4u
209
212
edits4u
1
213
001
1
1
007
2
2
00s
12
12
00sbaby
1
13
00sbaby
1
1
baby
97
98
011
1
1
011596
1
1
02h
1
1
toh
13
14
05
1
1
07
3
3
mt07
1
4
070shake
1
1
shakeitoff
23
24
0megaverse
1
1
0mniversal
1
1
0pium
1
1
1
94
94
1k
2
96
1v1
43
139
ej1
1
140
f1
346
486
formula1
187
673
hs1
4
677
jd1
1
678
jordan1
4
682
lan1
2
684
mv1
3
687
n1
1
688
night1
5
693
p1
2
695
peter1
14
709
prime1
1
710
round1
1
711
season1
15
726
sl1me
1
727
tlou1
12
739
ver1
2
741
williamsf1
1
742
10
15
15
10k
1
16
10sbabe
3
19
10years
1
20
ben10
6
26
fast10
2
28
lm10
3
31
mar10
1
32
tccnota10
2
34
top10
8
42
windows10
1
43
10000steps
2
2
steps
1
3
1000goaltoreach
7
7
100daysofcode
2
2
100k
4
4
daimonia100k
1
5
isaacs100keditcomp
1
6
kandi
14
20
the100
4
24
zexo100kcomp
2
26
100layers
3
3
the100
4
7
100million
1
1
million
3
4
the100
4
8
100solodateideas
2
2
cheapdateideas
2
4
datedayideas
1
5
falldateideas
1
6
seasonaldateideas
1
7
solobookdate
1
8
uniquedateideas
1
9
100years

In [180]:
clusters = sample_fifty[['Hashtag', 'Cluster', 'Count']]
clusters.to_csv('data/hashtags_clusters.csv', index=False)

In [181]:
clusters = pd.read_csv('data/hashtags_clusters.csv')

In [182]:
# order the clusters dataframe by the number of hashtags in each cluster
clusters_asc = pd.read_csv('data/hashtags_clusters.csv')
clusters_desc = pd.read_csv('data/hashtags_clusters.csv')

clusters_asc['Length'] = clusters_asc['Cluster'].str.split(',').str.len()
clusters_desc['Length'] = clusters_desc['Cluster'].str.split(',').str.len()

clusters_asc.sort_values(by=['Length'], inplace=True, ascending=True)
clusters_desc.sort_values(by=['Length'], inplace=True, ascending=False)
clusters_asc.drop(columns=['Length'], inplace=True)
clusters_desc.drop(columns=['Length'], inplace=True)

# clusters.head(10)


In [183]:
# get the row outside of the list
clusters_desc['Cluster'] = clusters_desc['Cluster'].str.replace('[', '')
clusters_desc['Cluster'] = clusters_desc['Cluster'].str.replace(']', '')
clusters_desc['Cluster'] = clusters_desc['Cluster'].str.replace(" ", '')

clusters_asc['Cluster'] = clusters_asc['Cluster'].str.replace('[', '')
clusters_asc['Cluster'] = clusters_asc['Cluster'].str.replace(']', '')
clusters_asc['Cluster'] = clusters_asc['Cluster'].str.replace(" ", '')

C:\Users\Test\AppData\Local\Temp\ipykernel_20796\1569565536.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  clusters_desc['Cluster'] = clusters_desc['Cluster'].str.replace('[', '')
C:\Users\Test\AppData\Local\Temp\ipykernel_20796\1569565536.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  clusters_desc['Cluster'] = clusters_desc['Cluster'].str.replace(']', '')
C:\Users\Test\AppData\Local\Temp\ipykernel_20796\1569565536.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  clusters_asc['Cluster'] = clusters_asc['Clus

In [184]:
indexes_to_drop = []
sub_indexes = []

for index, row in clusters_asc.iterrows():
    mask = clusters_desc['Cluster'].apply(lambda x: set(row['Cluster'].split(',')).issubset(set(x.split(','))))
    mask[index] = False  # Exclude the same index
    matching_clusters = clusters_desc[mask]

    if not matching_clusters.empty:
        print(row['Cluster'])
        print(index)
        sub_indexes.append(matching_clusters['Cluster'].index.tolist()[0])
        indexes_to_drop.append(index)

'koneledit'
18882
'nelsonedit'
23688
'nelsonandmurdock'
23687
'nekedbe'
23682
'neiperte'
23681
'neiljacksonedit'
23678
'neildruckmann'
23670
'negan'
23661
'neflix'
23659
'neferadenile'
23658
'needyfriends'
23656
'needycats'
23655
'neobatman'
23693
'neogotham'
23694
'neonatalkitten'
23696
'netflixlatam'
23725
'ness'
23717
'nepotismbaby'
23707
'nedfulmer'
23651
'naughtydog'
23597
'naturalmakeup'
23594
'natsufairytail'
23589
'nationalpancakeday'
23579
'nathanlane'
23568
'necrosword'
23646
'neckpainreliefn'
23643
'nda'
23628
'nctdream'
23624
'ncbd'
23621
'nds'
23630
'netos'
23736
'nets'
23737
'niaaep'
23884
'nflplayoffs'
23877
'neytiriedit'
23874
'nextlevel'
23869
'newyorker'
23864
'newton'
23857
'newstodayinuk'
23852
'newsies'
23846
'newshows2023'
23845
'nianal'
23890
'nickolasgalitzine'
23939
'nicknelson'
23937
'nickfink'
23922
'newschoolyear'
23843
'nickandjess'
23914
'nicholasbraunedit'
23899
'nicestcelebrities'
23896
'niceguy'
23894
'nickandcharlie'
23911
'nateheywood'
23558
'newschec

In [185]:
for index in sub_indexes:
    clusters.iloc[index, clusters.columns.get_loc('Count')] += clusters.iloc[indexes_to_drop[0], clusters.columns.get_loc('Count')]

In [186]:
for index in indexes_to_drop:
    clusters.drop(index, inplace=True)

In [187]:
clusters.to_csv('data/clean_hashtags/after_hashtags.csv', index=False)

In [ ]:
"""
Hierarchical Clustering
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

# Perform hierarchical clustering
linkage_matrix = linkage(similarity_matrix, method='ward')

# Create a dendrogram
dendrogram(linkage_matrix, labels=df['Hashtag'].tolist(), orientation='right')
plt.title('Hierarchical Clustering Dendrogram')
plt.show()
"""

"""
Determine Clusters
from scipy.cluster.hierarchy import fcluster

# Determine clusters
threshold = 0.2  # Adjust the threshold as needed
clusters = fcluster(linkage_matrix, threshold, criterion='distance')

# Add the cluster labels to the DataFrame
df['Cluster'] = clusters

# Display the DataFrame with cluster labels
print(df[['Hashtag', 'Cluster']])
"""